In [5]:
from tigeropen.common.consts import (Language,        # 语言
                                Market,           # 市场
                                BarPeriod,        # k线周期
                                QuoteRight)       # 复权类型
from tigeropen.tiger_open_config import TigerOpenClientConfig
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.quote.quote_client import QuoteClient
import os
import importlib

def get_client_config():
    """
    https://quant.itigerup.com/#developer 开发者信息获取
    """
    client_config = TigerOpenClientConfig()
    # 使用绝对路径读取私钥文件
    current_dir = os.path.dirname(os.path.abspath("./tiger-test"))
    private_key_path = os.path.join(current_dir, 'keys', 'private.pem')
    client_config.private_key = read_private_key(private_key_path)
    client_config.tiger_id = '20153826'
    # client_config.account = '20240803144534965'
    client_config.account = '498843'
    client_config.language = Language.zh_CN  #可选，不填默认为英语'
    # client_config.timezone = 'US/Eastern' # 可选时区设置
    return client_config

client_config = get_client_config()

# 随后传入配置参数对象来初始化QuoteClient
quote_client = QuoteClient(client_config)

# 完成初始化后，就可以调用quote_client方法来使用调用QuoteClient对象的get_stock_brief方法来查询股票行情了
# 此处以美国股票为例，关于其他支持的市场及标的类型，请参考文档的基本操作部分。
# 对于使用多台设备调用API的用户，需先调用grab_quote_permission进行行情权限的抢占，详情请见基本操作-行情类-通用-grab_quote_permission方法说明
permissions = quote_client.grab_quote_permission() 

In [ ]:
from tigeropen.trade.trade_client import TradeClient
from tigeropen.tiger_open_config import get_client_config
trade_client = TradeClient(client_config)

accounts = trade_client.get_managed_accounts()
# 查看第一个账户的相关属性
account1 = accounts[0]
print(account1.account)  # 账户号
print(account1.account_type)  # 账户分类(综合/模拟)
print(account1.capability)  # 账户能力(现金/保证金)


In [ ]:

trade_client = TradeClient(client_config)
portfolio_account = trade_client.get_prime_assets(base_currency='HKD')  # 可设置基准币种
print(portfolio_account)
# 查看账户属性示例
print(portfolio_account.account)   # 账户id
print(portfolio_account.segments['S'].buying_power)  # 证券账户购买力
print(portfolio_account.segments['S'].cash_balance)  # 证券账户现金值
print(portfolio_account.segments['S'].unrealized_pl)   # 浮动盈亏
print(portfolio_account.segments['S'].currency_assets['USD'].gross_position_value)  # 以美元计算的证券总价值
print(portfolio_account.segments['S'].currency_assets['HKD'].gross_position_value)  # 以港币计算的证券总价值
print(portfolio_account.segments['S'].currency_assets['HKD'].cash_balance)  # 以港币计算的现金值

print(portfolio_account.segments['C'].init_margin)  # 期货账户初始保证金
print(portfolio_account.segments['C'].maintain_margin)  # 期货账户维持保证金
print(portfolio_account.segments['C'].currency_assets['USD'].cash_balance)  # 期货账户以美元计算的现金值

In [ ]:
from tigeropen.common.util.contract_utils import stock_contract, option_contract, option_contract_by_symbol, \
    future_contract, war_contract_by_symbol, iopt_contract_by_symbol
from tigeropen.common.util.contract_utils import stock_contract
from tigeropen.common.util.order_utils import limit_order
from tigeropen.trade.trade_client import TradeClient
from tigeropen.tiger_open_config import get_client_config
# 美股
# contract = stock_contract(symbol='TIGR', currency='USD')

# 港股
contract = stock_contract(symbol='03033', currency='HKD')
order = limit_order(account=client_config.account, contract=contract, action='SELL', limit_price=5.32, quantity=200)

# 下单
oid = trade_client.place_order(order)

print(order)
# >>> Order({'account': '111111', 'id': 2498911111111111111, 'order_id': None, 'parent_id': None, 'order_time': None, 'reason': None, 'trade_time': None, 'action': 'BUY', 'quantity': 1, 'filled': 0, 'avg_fill_price': 0, 'commission': None, 'realized_pnl': None, 'trail_stop_price': None, 'limit_price': 0.1, 'aux_price': None, 'trailing_percent': None, 'percent_offset': None, 'order_type': 'LMT', 'time_in_force': None, 'outside_rth': None, 'order_legs': None, 'algo_params': None, 'secret_key': None, 'contract': AAPL/STK/USD, 'status': 'NEW', 'remaining': 1})
print(order.status)  # 订单状态
print(order.reason)  # 若下单失败，reason为失败原因